In [1]:
import numpy as np
import pandas as pd

In [2]:
#Scrapping data from wikipedia

import requests
from lxml import html

pageContent=requests.get(
     'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
)
tree = html.fromstring(pageContent.content)
torontoTable = tree.xpath('//*[@id="mw-content-text"]/div/table[1]/tbody//text()')

In [3]:
#continue preprocess wikipedia data with XPATH

counter = 0
torontoArray =[]
torontoItem =""

for data in torontoTable:
    if data.strip() != '' and data.strip() != 'Postal code' and data.strip() != 'Borough' and data.strip() != 'Neighborhood' :
        if counter%3 == 0:
            if data.strip()[0] != 'M':
                continue
            torontoArray.append(torontoItem)
            torontoItem=data.strip()
        elif data.strip() != 'Not assigned':
            torontoItem =   torontoItem.strip() + "|" + data.strip().replace(" /", ",")
        else :
            torontoItem =   torontoItem.strip()  + "|" + data.strip().replace(" /", ",")
            counter = counter +1
        counter = counter +1
       
    
len(torontoArray)

180

In [ ]:
#Adding Latitude and Longtitude

import geocoder
torontoList = []

for item in torontoArray:
    if 'Not assigned' in item :
        continue
    else : 
        if item != '' :
            ItemValue = str(item).split("|")
            stringAddress = ItemValue[0] +", Toronto, Ontario"
            coordinates = geocoder.arcgis(stringAddress).latlng
            ItemValue.append(coordinates[0])
            ItemValue.append(coordinates[1])
            torontoList.append(ItemValue) 
            
torontoList         

In [ ]:
#Adding into Pandas Dataframe
df = pd.DataFrame(torontoList, columns =['Postal code', 'Borough','Neighborhood', 'Latitude' , 'Longitude']) 

In [ ]:
df.shape

In [ ]:
df.head(12)

In [ ]:
from geopy.geocoders import Nominatim
import folium

address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
DisplayMap_folium = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'],  df['Neighborhood']):
    label = f'neighborhood: {neighborhood}'
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='#000fff',
        fill=True,
        fill_color='#000fff',
        fill_opacity=0.5,
        parse_html=False).add_to(DisplayMap_folium)  
    
DisplayMap_folium

In [ ]:
# foursquare information
CLIENT_ID = '0DPTJZXTG0VRTPDCOW0V3KNG32M40D1XWWBVYFF0VNYYG2PM' # Foursquare ID
CLIENT_SECRET = 'WWEN1VYCGVR4E2FOTQRPFYM0I03MIDXF2DK1NEMSEXDIJN2B' # Foursquare Secret
VERSION = '20200408'
RADIUS = 200
LIMIT = 100

for name, lat, lng in zip(  df['Neighborhood'] , df['Latitude'], df['Longitude']):
        venues_list=[]
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            RADIUS, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
            df['Neighborhood'], 
            df['Latitude'], 
            df['Longitude'], 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

all_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
all_venues.columns = ['Neighborhood',                   
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']